<a href="https://colab.research.google.com/github/RMDircio/DS-Unit-4-Sprint-3-Deep-Learning/blob/main/module1-rnn-and-lstm/Regina_Dircio_DSPT5_431_RNN_and_LSTM_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [1]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

import pandas as pd
import numpy as np
import requests
import random
import sys
import os



In [2]:
url = "https://www.gutenberg.org/files/100/100-0.txt"

r = requests.get(url)
r.encoding = r.apparent_encoding
data = r.text
data = data.split('\r\n')
toc = [l.strip() for l in data[44:130:2]]
# Skip the Table of Contents
data = data[135:]

# Fixing Titles
toc[9] = 'THE LIFE OF KING HENRY V'
toc[18] = 'MACBETH'
toc[24] = 'OTHELLO, THE MOOR OF VENICE'
toc[34] = 'TWELFTH NIGHT: OR, WHAT YOU WILL'

locations = {id_:{'title':title, 'start':-99} for id_,title in enumerate(toc)}

# Start 
for e,i in enumerate(data):
    for t,title in enumerate(toc):
        if title in i:
            locations[t].update({'start':e})
            

df_toc = pd.DataFrame.from_dict(locations, orient='index')
df_toc['end'] = df_toc['start'].shift(-1).apply(lambda x: x-1)
df_toc.loc[42, 'end'] = len(data)
df_toc['end'] = df_toc['end'].astype('int')

df_toc['text'] = df_toc.apply(lambda x: '\r\n'.join(data[ x['start'] : int(x['end']) ]), axis=1)

In [3]:
#Shakespeare Data Parsed by Play
df_toc.head()

,title,start,end,text
0,THE TRAGEDY OF ANTONY AND CLEOPATRA,-99,14379,
1,AS YOU LIKE IT,14380,17171,AS YOU LIKE IT\r\n\r\n\r\nDRAMATIS PERSONAE.\r...
2,THE COMEDY OF ERRORS,17172,20372,THE COMEDY OF ERRORS\r\n\r\n\r\n\r\nContents\r...
3,THE TRAGEDY OF CORIOLANUS,20373,30346,THE TRAGEDY OF CORIOLANUS\r\n\r\nDramatis Pers...
4,CYMBELINE,30347,30364,CYMBELINE.\r\nLaud we the gods;\r\nAnd let our...


In [4]:
# number of plays
len(df_toc)

43

In [5]:
import re

text = " ".join(df_toc['text']) # combine all the plays
text = re.sub("[^A-Za-z ]", " ", text)
text = re.sub(" +", " ", text)
# words = list(set(text.split(" "))) # seperated words by space
words = list(set(text))

# Lookup Tables
words_int = {w:i for i, w in enumerate(words)} 
int_words = {i:w for i, w in enumerate(words)} 

In [6]:
len(words)

53

In [7]:
words[0], words[1], words[2], words[3]

('J', 'z', 'C', 'w')

In [10]:
# Create the sequence data
# set up input data and labels

maxlen = 30
step = 5

encoded = [words_int[w] for w in words]

sequences = [] # Each element is 20 chars long
next_word = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    
    sequences.append(encoded[i : i + maxlen]) # training data
    next_word.append(encoded[i + maxlen]) # labels
    
print('sequences: ', len(sequences))

sequences:  5


In [12]:
sequences[0] # mapped numbers = words

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29]

In [13]:
next_word[0], int_words[next_word[0]]

(30, 't')

In [14]:
# Create x & y

# these are two empty arrays
x = np.zeros((len(sequences), maxlen, len(words)), dtype=np.bool)
y = np.zeros((len(sequences),len(words)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for w, word in enumerate(sequence):
        x[i,w,word] = 1
        
    y[i, next_word[i]] = 1

In [15]:
x.shape, y.shape

((5, 30, 53), (5, 53))

In [16]:
# building model: single LSTM

# no embedding layer
# train as if cateogorical/classifcation
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(words))))
model.add(Dense(len(words), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [17]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)



In [18]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(words)))
        for w, word in enumerate(sentence):
            x_pred[0, w, words_int[word]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_word = int_words[next_index]
        
        sentence = sentence[1:] + next_word
        
        sys.stdout.write(next_word)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [19]:
# fit the model

model.fit(x, y,
          batch_size=32,
          epochs=10,
          callbacks=[print_callback])

Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 3.9744
----- Generating text after Epoch: 0
----- Generating with seed: "III Tarsus A room in Cleon s h"
III Tarsus A room in Cleon s hfzgJ KGcG  IBivXrdzrCYjhEsMzGxbzhnBItjpoJhJwr xzvnSyNPWN qCRqYyxQlykWwsSv UWIAKdOAGVqqmuxfGsOkojHiZpvxiI NiRWzcdBxwzpqYCdDThpqJfwusFXTsViGRxWLdhJmLdyOOgyLaPemHVSaoRbkhBpZMFhAfjGJbrXApvUiHSSrCTmXFUfnPIKcoRHNeJuUXHoXtbrjSUkbUTazXuHjPkCgPELhTeLJCeMlSiMrTWxhcWnIeWkSezpeHY kRlIpiEahwBTHvAoyCSmMh pnUgQlDdVCagNRzYTphtmHJXEHYeBMWNubjvyeqTciwjzyWULPRfBKNtogSCqNNfgcoCJdbjwCBJrfByX ntJWNmIzjkSXdhVIkgjbqGfytr
1/1 [==============================] - 17s 17s/step - loss: 3.9744
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 3.9441
----- Generating text after Epoch: 1
----- Generating with seed: "ll let em have kind admittance"
ll let em have kind admittanceXaFMyfRmjxXrpVqUBEQgMPyXgZMTwywLcBcGOKmrOmXGmNXFOKwCxGs jQaqUaaJNYDmyZANmMwzAe VohxZpY veDXXirWVYxTSNmCEVPUKxOXzBQffzKSZdMkovxDVlc

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN